In [4]:
import monai
import torch
import torch.nn as nn

In [2]:
model = monai.networks.nets.resnet18(spatial_dims=3, n_input_channels=3, num_classes=2)
encoder_layers = list(model.children())[:-1]
encoder_layers.append(torch.nn.Flatten(start_dim=1, end_dim=-1))                
encoder = nn.Sequential(*encoder_layers)

In [4]:
encoder(torch.randn(2,3,64,64,64)).shape


torch.Size([2, 512])

In [6]:
torch.sum([torch.tensor([1,2,3]), torch.tensor([4,5,6])])

TypeError: sum(): argument 'input' (position 1) must be Tensor, not list